In [ ]:
import os
import warnings

import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.optim as optim

from hmmlearn import hmm

# Suppress warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convolutional Autoencoder Structure:

In [ ]:
# Define the Convolutional Autoencoder (CAE) architecture
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()

        self.ls = 32

        # Define separate encoders for each signal
        self.ecg_encoder = nn.Sequential(
            nn.Conv1d(1, 8, kernel_size=20, stride=10, padding=1),
            nn.ReLU(),
            nn.Conv1d(8, self.ls, kernel_size=10, stride=5, padding=1),
        )
        self.rsp_encoder = self.ecg_encoder
        self.eda_tonic_encoder = self.ecg_encoder
        self.eda_phasic_encoder = self.ecg_encoder

        # Fully connected layer to compress the latent space
        self.fc = nn.Linear(128, self.ls)

        # Fully connected layer to decompress the latent space
        self.fc_decoded = nn.Linear(self.ls, 128)

        # Define separate decoders for each signal
        self.ecg_decoder = nn.Sequential(
            nn.ConvTranspose1d(self.ls, 8, kernel_size=10, stride=5, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(8, 1, kernel_size=20, stride=10, padding=1, output_padding=2),
            nn.Sigmoid(),
        )
        self.rsp_decoder = self.ecg_decoder
        self.eda_tonic_decoder = self.ecg_decoder
        self.eda_phasic_decoder = self.ecg_decoder

    def encode(self, x):
        # Split input by channel for independent processing
        ecg = x[:, 0, :].unsqueeze(1)
        rsp = x[:, 1, :].unsqueeze(1)
        eda_tonic = x[:, 2, :].unsqueeze(1)
        eda_phasic = x[:, 3, :].unsqueeze(1)

        # Encode each signal independently
        ecg_encoded = self.ecg_encoder(ecg)
        rsp_encoded = self.rsp_encoder(rsp)
        eda_tonic_encoded = self.eda_tonic_encoder(eda_tonic)
        eda_phasic_encoded = self.eda_phasic_encoder(eda_phasic)

        # Concatenate the latent representations along the last dimension
        latent_space = torch.cat(
            (ecg_encoded, rsp_encoded, eda_tonic_encoded, eda_phasic_encoded), dim=1
        )

        latent_space = latent_space.permute(0, 2, 1)

        # Compress the latent space
        latent_space = self.fc(latent_space)

        return latent_space

    def decode(self, latent_space):
        # Decompress the latent space
        latent_space = self.fc_decoded(latent_space)
        latent_space = latent_space.permute(0, 2, 1)

        # Split latent space back into separate channels
        ecg_latent, rsp_latent, eda_tonic_latent, eda_phasic_latent = torch.split(
            latent_space, self.ls, dim=1
        )
        # Decode each signal independently
        ecg_decoded = self.ecg_decoder(ecg_latent)
        rsp_decoded = self.rsp_decoder(rsp_latent)
        eda_tonic_decoded = self.eda_tonic_decoder(eda_tonic_latent)
        eda_phasic_decoded = self.eda_phasic_decoder(eda_phasic_latent)

        # Concatenate the decoded signals to form the output
        reconstructed = torch.cat(
            (ecg_decoded, rsp_decoded, eda_tonic_decoded, eda_phasic_decoded), dim=1
        )

        return reconstructed

    def forward(self, x):
        latent_space = self.encode(x)
        reconstructed = self.decode(latent_space)
        return reconstructed

In [ ]:
learning_rate = 0.001

# Train Autoencoders to Each Participants' Baseline Data

In [ ]:
segment_size = "1s"
step_size = "0.001s"

In [ ]:
# loop through baseline data
for file in os.listdir("./Physiological Preprocessed/Exp2"):
    participant = file.split("_")[0]
    if "baseline" not in file:
        continue
    elif os.path.exists(f"./Convolutional Autoencoder Models/Takeover Time/{participant}_model.pth"):
        print(f"Model for {file} already exists")
        continue

    print(f"Loading {participant} data")
    print(f"-" * 50)

    # load data
    physiological_data = pd.read_csv(f"./Physiological Preprocessed/Exp2/{file}", usecols=["Time", "ECG_Clean", "RSP_Clean", "EDA_Tonic", "EDA_Phasic"])
    physiological_data["Time"] = pd.to_timedelta(physiological_data["Time"])
    physiological_data.set_index("Time", inplace=True)

    print(f"Processing participant {participant} data")

    # Normalize the data
    scalar = MinMaxScaler()
    data = scalar.fit_transform(physiological_data)
    physiological_data = pd.DataFrame(data, columns=physiological_data.columns, index=physiological_data.index)

    # Split the data into sliding windows
    X = []
    len_segment = pd.Timedelta(segment_size) / pd.Timedelta(step_size)
    while len(physiological_data) > 0:
        start_index = physiological_data.index[0]
        end_index = start_index + pd.Timedelta(segment_size)
        segment = physiological_data[:end_index]
        physiological_data = physiological_data[end_index:]

        if len(segment) > len_segment:
            length = len(segment) - len_segment
            segment = segment.drop(segment.tail(int(length)).index)

        if len(segment) == len_segment:
            X.append(segment.to_numpy())

    X = np.stack(X)

    # Convert to PyTorch tensors
    X = torch.tensor(X, dtype=torch.float32).to(device)

    # Initialize the model
    model = ConvAutoencoder().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)

    # Training parameters
    batch_size = 4
    num_epochs = 300

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        for i in range(0, len(X), batch_size):

            # Get the batch and reshape it for Conv1d (batch, channels, sequence_length)
            batch = X[i : i + batch_size].permute(0, 2, 1)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(batch)

            # Calculate loss
            loss = criterion(outputs, batch)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(X):.4f}")

    # Save the model
    if not os.path.exists("./Convolutional Autoencoder Models/Takeover Time"):
        os.makedirs("./Convolutional Autoencoder Models/Takeover Time")
    torch.save(model.state_dict(), f"./Convolutional Autoencoder Models/Takeover Time/{participant}_model.pth")

---
## Constructing Observations

In [ ]:
obstacles = ["Deer", "Cone", "Frog", "Can"]

processed_physio_folder_path = "./Physiological Preprocessed/"

exp2_folder_path = processed_physio_folder_path + "Exp2"

exp2_takeover_times = pd.read_csv("./AdVitam/Exp2/Preprocessed/Physio and Driving/timestamps_obstacles.csv")
exp2_takeover_times.iloc[:, 2:] = exp2_takeover_times.iloc[:, 2:].apply(pd.to_timedelta, unit="s")
exp2_takeover_times["subject_id"] = exp2_takeover_times["subject_id"].apply(lambda x: x.split("T")[0] + "T" + x.split("T")[1].zfill(2))
exp2_takeover_times["subject_id"] = exp2_takeover_times["subject_id"].astype(str)
exp2_takeover_times.drop(columns=["label_st"], inplace=True)
exp2_takeover_times.sort_values(by=["subject_id"], inplace=True)

for column in exp2_takeover_times.columns:
    if "TrigObs" in column:
        exp2_takeover_times = exp2_takeover_times.rename(
            columns={column: column.replace("TrigObs", "") + "TOR"}
        )
    elif "RepObs" in column:
        exp2_takeover_times = exp2_takeover_times.rename(
            columns={column: column.replace("RepObs", "Response")}
        )

for obstacle in obstacles:
    exp2_takeover_times["TOT" + obstacle] = (
        exp2_takeover_times["Response" + obstacle] - exp2_takeover_times[obstacle + "TOR"]
    )

exp2_takeover_times

In [ ]:
# if TOT is greater than 10 s, or less than 0 s, change to NaT
for obstacle in obstacles:
    exp2_takeover_times.loc[
        (exp2_takeover_times["TOT" + obstacle] > pd.Timedelta("10s"))
        | (exp2_takeover_times["TOT" + obstacle] < pd.Timedelta("0s")),
        "TOT" + obstacle,
    ] = pd.NaT


# print max and min takeover times
for obstacle in obstacles:
    print(obstacle)
    print("Max takeover time: ", exp2_takeover_times["TOT" + obstacle].max())
    print("Min takeover time: ", exp2_takeover_times["TOT" + obstacle].min())
    print("Mean takeover time: ", exp2_takeover_times["TOT" + obstacle].mean())
    print("Median takeover time: ", exp2_takeover_times["TOT" + obstacle].median())
    print("Standard deviation of takeover time: ", exp2_takeover_times["TOT" + obstacle].std())
    print("\n")

In [ ]:
def collect_observations(phsyiological_data_folder, include_latent_space, jump, takeover_times, tot):
    """
    Create the observations for the slow and fast takeover times.

    Args:
        phsyiological_data_dictionary (dict): A dictionary containing the segmented physiological data files.
        takeover_times (pd.DataFrame): A DataFrame containing the takeover times.
        driver_demographic_data (pd.DataFrame): A DataFrame containing the driver demographic data.
        window_length (int): The length of the window in minutes.
        window_step (int): The step size for the window

    Returns:
        list: A list of observations for the slow takeover times.
        list: A list of observations for the fast takeover times.
    """

    slow_observations = []
    fast_observations = []

    for file in os.listdir(phsyiological_data_folder):
        # Split the file name into the participant and period
        f = file.split("_")
        participant = f[0]
        period = f[1].split(".")[0]

        if period != "driving":
            continue

        # print(f"Processing {participant} data")

        # Physiological data
        physio = pd.read_csv(phsyiological_data_folder + "/" + file, usecols=["Time", "ECG_Clean", "RSP_Clean", "EDA_Tonic", "EDA_Phasic"])
        physio["Time"] = pd.to_timedelta(physio["Time"])
        physio.set_index("Time", inplace=True)

        # Initialize the model
        model = ConvAutoencoder().to(device)
        criterion = nn.MSELoss()
        model.load_state_dict(torch.load(f"./Convolutional Autoencoder Models/Takeover Time/{participant}_model.pth"))
        model.eval()

        scalar = MinMaxScaler()
        data = scalar.fit_transform(physio)
        physio = pd.DataFrame(data, columns=physio.columns, index=physio.index)

        # Takeover Times
        participant_takeover_times = takeover_times[
            takeover_times["subject_id"] == participant
        ].copy()

        # convert every value to timedelta
        participant_takeover_times.iloc[:, 1:] = participant_takeover_times.iloc[:, 1:].apply(
            pd.to_timedelta, args=("s",), errors="coerce"
        )

        for obstacle in obstacles:
            takeover_time = participant_takeover_times[f"TOT{obstacle}"].values[0]

            if pd.isna(takeover_time) or pd.isnull(takeover_time):
                continue

            obstacle_trigger_time = pd.to_timedelta(participant_takeover_times[f"{obstacle}TOR"].values[0], unit="s")

            if pd.isnull(obstacle_trigger_time):
                continue

            # Get the physiological data for the period
            obstacle_physio = physio.loc[obstacle_trigger_time - pd.Timedelta(seconds=4):obstacle_trigger_time + pd.Timedelta(seconds=8)]

            step_size = "0.001s"
            observations = []
            len_segment = pd.Timedelta(segment_size) / pd.Timedelta(step_size)
            while len(obstacle_physio) > 0:
                start_index = obstacle_physio.index[0]
                end_index = start_index + pd.Timedelta(segment_size)
                segmented_signal = obstacle_physio[:end_index]
                obstacle_physio = obstacle_physio[start_index + pd.Timedelta(jump) :]

                if len(segmented_signal) > len_segment:
                    length = len(segmented_signal) - len_segment
                    segmented_signal = segmented_signal.drop(segmented_signal.tail(int(length)).index)

                if len(segmented_signal) == len_segment:
                    segmented_signal = torch.tensor(segmented_signal.to_numpy(), dtype=torch.float32).to(device)
                    with torch.no_grad():
                        latent_space = model.encode(segmented_signal.permute(1, 0).unsqueeze(0))
                        reconstructed_signal = model.decode(latent_space).squeeze(0).permute(1, 0)
                    loss = criterion(reconstructed_signal, segmented_signal)
                    if include_latent_space:
                        pca_signal = PCA(n_components=1).fit_transform(latent_space.cpu().detach().numpy().squeeze()).squeeze()
                        segmented_observations = np.concatenate((pca_signal, [loss.item()]))
                    else:
                        segmented_observations.append(loss.item())

                    observations.append(segmented_observations)

            observations = np.stack(observations)

            if len(observations) == 0:
                continue

            # print(numpy_data)
            if takeover_time > pd.to_timedelta("0 days 00:00:0" + tot):
                slow_observations.append(observations)

            else:
                fast_observations.append(observations)

    return slow_observations, fast_observations

In [ ]:
def accuracy(
    slow_observations,
    fast_observations,
    n_components_slow,
    n_components_fast,
    n_mix_slow,
    n_mix_fast,
    covariance_type,
    include_latent_space
):
    iterations = 100

    accuracies = []
    true_positives_list = []
    false_positives_list = []
    true_negatives_list = []
    false_negatives_list = []

    for i in range(iterations):
        if i % 10 == 0:
            print(f"Iteration: {i}")

        # split the data
        slow_observations_train, slow_observations_test = train_test_split(
            slow_observations, test_size=0.3
        )
        fast_observations_train, fast_observations_test = train_test_split(
            fast_observations, test_size=0.3
        )

        # concatenate the observations
        X_slow = None
        X_slow_lengths = []
        for data in slow_observations_train:
            if X_slow is None:
                X_slow = data
            else:
                X_slow = np.concatenate((X_slow, data))
            X_slow_lengths.append(len(data))

        X_fast = None
        X_fast_lengths = []
        for data in fast_observations_train:
            if X_fast is None:
                X_fast = data
            else:
                X_fast = np.concatenate((X_fast, data))
            X_fast_lengths.append(len(data))

        # initialize and fit the models
        slow_model = hmm.GMMHMM(
            n_components=n_components_slow, n_mix=n_mix_slow, covariance_type=covariance_type
        )
        fast_model = hmm.GMMHMM(
            n_components=n_components_fast, n_mix=n_mix_fast, covariance_type=covariance_type
        )

        # fit the models
        if not include_latent_space:
            X_slow = X_slow.reshape(-1, 1)
            X_fast = X_fast.reshape(-1, 1)
        slow_model.fit(X_slow, X_slow_lengths)
        fast_model.fit(X_fast, X_fast_lengths)

        # score the models
        accuracy = 0
        tp = 0
        fp = 0
        tn = 0
        fn = 0

        for _, observation in enumerate(slow_observations_test):
            if not include_latent_space:
                observation = observation.reshape(-1, 1)

            if slow_model.score(observation) > fast_model.score(observation):
                accuracy += 1
                tn += 1
            else:
                fn += 1

        for _, observation in enumerate(fast_observations_test):
            if not include_latent_space:
                observation = observation.reshape(-1, 1)

            if fast_model.score(observation) > slow_model.score(observation):
                accuracy += 1
                tp += 1
            else:
                fp += 1

        accuracy = accuracy / (len(slow_observations_test) + len(fast_observations_test))
        accuracies.append(accuracy)

        true_positives_list.append(tp)
        false_positives_list.append(fp)
        true_negatives_list.append(tn)
        false_negatives_list.append(fn)

        if i % 10 == 0:
            print(f"Accuracy: {accuracy}")
            print(f"True Positives: {tp}")
            print(f"False Positives: {fp}")
            print(f"True Negatives: {tn}")
            print(f"False Negatives: {fn}")

    return (
        accuracies,
        true_positives_list,
        false_positives_list,
        true_negatives_list,
        false_negatives_list,
    )

In [ ]:
tot = "3"
include_latent_space = [True, False]
jump_size = ["1s", "500ms", "100ms"]
n_components_slow = [1, 2, 3, 4]
n_mix_slow = [1, 2, 3, 4]
n_components_fast = [1, 2, 3, 4]
n_mix_fast = [1, 2, 3, 4]
covariance_types = ["full", "diag", "spherical"]

In [ ]:
for include_latent in include_latent_space:
    for jump in jump_size:
        print(f"Collecting observations every {jump} {'with latent space' if include_latent else ''}")
        slow_observations, fast_observations = collect_observations(exp2_folder_path, include_latent, jump, exp2_takeover_times, tot)
        print(f"Collected {len(slow_observations)} slow takeover times and {len(fast_observations)} fast takeover times")

        for n_slow in n_components_slow:
            for m_slow in n_mix_slow:
                for n_fast in n_components_fast:
                    for m_fast in n_mix_fast:
                        for cov in covariance_types:

                            # Check if the results already exist
                            if os.path.exists("results-tot-ehmm.csv"):
                                results_df = pd.read_csv("results-tot-ehmm.csv")
                                results = results_df[
                                        (results_df["TOT"] == int(tot)) 
                                        & (results_df["Latent Space"] == include_latent)
                                        & (results_df["Jump"] == jump)
                                        & (results_df["Components Slow"] == n_slow)
                                        & (results_df["Mixtures Slow"] == m_slow)
                                        & (results_df["Components Fast"] == n_fast)
                                        & (results_df["Mixtures Fast"] == m_fast)
                                        & (results_df["Covariance"] == cov)
                                    ]
                                if not results.empty:
                                    print("Results already exist for these parameters.")
                                    continue

                            print("-------------------------------------------------")
                            print(f"Slow Components: {n_slow}, Slow Mixtures: {m_slow}, Fast Components: {n_fast}, Fast Mixtures: {m_fast}, Covariance: {cov}")
                            try:
                                accuracies, true_positives_list, false_positives_list, true_negatives_list, false_negatives_list = accuracy(slow_observations, fast_observations, n_slow, n_fast, m_slow, m_fast, cov, include_latent )

                                # Accuracy
                                print(f"Average Accuracy: {np.mean(accuracies)}")
                                print(f"Standard Deviation: {np.std(accuracies)}")
                                print(f"Max Accuracy: {np.max(accuracies)}")
                                print(f"Min Accuracy: {np.min(accuracies)}")

                                # Find the index of the max accuracy
                                max_accuracy_index = accuracies.index(np.max(accuracies))
                                tp = true_positives_list[max_accuracy_index]
                                print(f"True Positives: {tp}")
                                fp = false_positives_list[max_accuracy_index]
                                print(f"False Positives: {fp}")
                                tn = true_negatives_list[max_accuracy_index]
                                print(f"True Negatives: {tn}")
                                fn = false_negatives_list[max_accuracy_index]
                                print(f"False Negatives: {fn}")

                                # check  if any of the values are zero
                                if tp + fp == 0 or tp + fn == 0:
                                    precision = 0
                                    recall = 0
                                    f1_score = 0
                                else:
                                    # Precision, Recall, and F1 Score
                                    precision = tp / (tp + fp)
                                    recall = tp / (tp + fn)
                                    f1_score = 2 * precision * recall / (precision + recall)
                                print(f'Precision: {precision}')
                                print(f'Recall: {recall}') 
                                print(f'F1 Score: {f1_score}')
                                print("-------------------------------------------------")
                                print()
                                results = [tot, include_latent, jump, n_slow, m_slow, n_fast, m_fast, cov, np.mean(accuracies), np.std(accuracies), np.max(accuracies), np.min(accuracies), tp, fp, tn, fn, precision, recall, f1_score]
                            except:
                                results = [tot, include_latent, jump, n_slow, m_slow, n_fast, m_fast, cov, None, None, None, None, None, None, None, None, None, None, None]

                            results_columns = ["TOT", "Latent Space", "Jump", "Components Slow", "Mixtures Slow", "Components Fast", "Mixtures Fast", "Covariance", "Mean Accuracy", "Standard Deviation", "Max Accuracy", "Min Accuracy", "True Positives", "False Positives", "True Negatives", "False Negatives", "Precision", "Recall", "F1 Score"]

                            if os.path.exists("results-tot-ehmm.csv"):
                                results_df = pd.read_csv("results-tot-ehmm.csv")
                                results_df = pd.concat([results_df, pd.DataFrame([results], columns=results_columns)])
                                results_df.to_csv("results-tot-ehmm.csv", index=False)
                            else:
                                pd.DataFrame([results], columns=results_columns).to_csv("results-tot-ehmm.csv", index=False)
